# AztecPlanner API 

Run this command first
```
uvicorn api_server:app --reload --host 127.0.0.1 --port 8000
```

If you didn't install dependencies yet, install them first: `python -m pip install -r requirements.txt`.

In [2]:
import time
from pprint import pprint
import requests

# Base URL for the API server
BASE_URL = "http://127.0.0.1:8000"

In [27]:
# Health check
def health():
    r = requests.get(f"{BASE_URL}/health")
    r.raise_for_status()
    return r.json()

pprint(health())

{'chroma_collections': ['allClasses', 'rateMyProfClasses'],
 'lite_llm': True,
 'status': 'ok'}


In [3]:
# List courses (optionally with prefix)
def list_courses(prefix=None):
    params = {}
    if prefix is not None:
        params['prefix'] = prefix
    r = requests.get(f"{BASE_URL}/courses", params=params)
    r.raise_for_status()
    return r.json()

courses = list_courses()
print(f"Got {len(courses)} courses (showing first 5):")
pprint(courses[:5])

Got 48 courses (showing first 5):
[{'code': 'CS 100',
  'description': 'Capabilities and applications of computers. Algorithmic '
                 'problem-solving methods and computer programming. Using '
                 'computers to examine questions from other fields of study. '
                 'Practical and theoretical limits to computation. Machine '
                 'intelligence and heuristic problem solving. Social and legal '
                 'impact of computers',
  'detail_url': 'https://catalog.sdsu.edu/preview_course_nopop.php?catoid=11&coid=80612',
  'general_education': 'Math/Quantitative Reasoning [2]',
  'grading_method': 'LCR: Letter Grade with Cr/NC available. The grading '
                    'default for the class will be letter grade, but students '
                    'can opt to take it for Cr/NC',
  'max_credits': '3',
  'name': 'Computer Science Principles',
  'notes': None,
  'prereqs': 'Satisfaction of the SDSU Mathematics/Quantitative Reasoning '
      

In [4]:
# Search courses by free text
def search_courses(q):
    r = requests.get(f"{BASE_URL}/courses/search", params={'q': q})
    r.raise_for_status()
    return r.json()

print('Search for algorithms:')
pprint(search_courses('algorithms')[:5])

Search for algorithms:
[{'code': 'CS 460',
  'description': 'Algorithms for solving frequently occurring problems. '
                 'Analysis techniques, divide and conquer algorithms with '
                 'applications, graph problems, greedy algorithms. '
                 'Introduction to NP complete problems. Formerly numbered: CS '
                 '560',
  'detail_url': 'https://catalog.sdsu.edu/preview_course_nopop.php?catoid=11&coid=84987',
  'general_education': None,
  'grading_method': 'LTR: Letter Graded. The class will be offered for letter '
                    'grade with no option to take it for Cr/NC',
  'max_credits': '3',
  'name': 'Algorithms',
  'notes': None,
  'prereqs': 'CS 210 and MATH 245',
  'restrictions': None,
  'typically_offered': 'Fall/Spring',
  'units': '3'}]


In [5]:
# Get a single course by code
def get_course(code):
    r = requests.get(f"{BASE_URL}/courses/{code}")
    if r.status_code == 404:
        return None
    r.raise_for_status()
    return r.json()

code = 'CS 240'
print(f"Course {code}:")
pprint(get_course(code))

Course CS 240:
{'code': 'CS 240',
 'description': 'Organization and assembly language to include CPU, logic '
                'circuits, and memory. Data representation, interrupts, '
                'looping and addressing techniques, macros, and traps',
 'detail_url': 'https://catalog.sdsu.edu/preview_course_nopop.php?catoid=11&coid=84714',
 'general_education': None,
 'grading_method': 'LTR: Letter Graded. The class will be offered for letter '
                   'grade with no option to take it for Cr/NC',
 'max_credits': '3',
 'name': 'Computer Organization',
 'notes': None,
 'prereqs': 'Credit or concurrent registration in CS 160',
 'restrictions': 'Not open to students with credit in CS 237.',
 'typically_offered': 'Fall/Spring',
 'units': '3'}


In [23]:
# Chroma batch add + query (may fail if chromadb isn't configured)
def chroma_add_batch(collection, documents):
    r = requests.post(f"{BASE_URL}/chroma/add_batch", json={'collection': collection, 'documents': documents})
    r.raise_for_status()
    return r.json()

def chroma_query(collection, query, n_results=3):
    r = requests.post(f"{BASE_URL}/chroma/query", json={'collection': collection, 'query': query, 'n_results': n_results})
    r.raise_for_status()
    return r.json()

try:
    import sys
    sys.path.append('..')
    import json
    print('Loading courses from utils/data/sdsu_cs_courses.json...')
    with open('../utils/data/sdsu_cs_courses.json','r',encoding='utf-8') as f:
        courses = json.load(f)
    documents = [str(c) for c in courses]
    print(f'Adding {len(documents)} courses to allClasses via batch endpoint...')
    add_resp = chroma_add_batch('allClasses', documents)
    pprint(add_resp)
    time.sleep(0.5)
    print('Querying for similar documents...')
    q = chroma_query('allClasses', 'data structures', n_results=2)
    pprint(q)
except Exception as e:
    print('Chroma endpoints failed:', e)

Loading courses from utils/data/sdsu_cs_courses.json...
Adding 48 courses to allClasses via batch endpoint...
Chroma endpoints failed: 400 Client Error: Bad Request for url: http://127.0.0.1:8000/chroma/add_batch


In [9]:
# LLM call (will return 503 if LLM is not configured)
def call_llm(message, system_prompt=''):
    r = requests.post(f"{BASE_URL}/llm", json={'message': message, 'system_prompt': system_prompt})
    if r.status_code == 503:
        return {'error': 'LLM not configured on server (503)'}
    r.raise_for_status()
    return r.json()

print('Calling LLM (if configured) with a simple prompt...')
response = call_llm('Hello from the test notebook')
pprint(response)

Calling LLM (if configured) with a simple prompt...
{'reply': "Hello from the main chat! It looks like we're trying out a "
          'connection between a test notebook and this chat interface. How can '
          'I assist you today?'}


In [5]:
def rag_query(message):
    r = requests.post(f"{BASE_URL}/rag", json={'message': message})
    if r.status_code == 503:
        return {'error': 'RAG not configured on server (503)'}
    r.raise_for_status()
    return r.json()

print('Calling RAG (if configured) with a question about data structures...')
rag_response = rag_query('What are some classes that cover data structures?')
pprint(rag_response)

Calling RAG (if configured) with a question about data structures...
{'reply': 'Based on the provided context, the following classes cover data '
          'structures:\n'
          '\n'
          '1. CS 210 - Data Structures \n'
          '2. CS 305 - Data Structures for Scientists and Engineers',
 'retrieved_documents': ["{'code': 'CS 210', 'name': 'Data Structures', "
                         "'detail_url': "
                         "'https://catalog.sdsu.edu/preview_course_nopop.php?catoid=11&coid=80619', "
                         "'units': '3', 'general_education': None, "
                         "'grading_method': 'LTR: Letter Graded. The class "
                         'will be offered for letter grade with no option to '
                         "take it for Cr/NC', 'prereqs': 'CS 160', "
                         "'restrictions': None, 'description': "
                         "'Representations and operations on basic data "
                         'structures. Arrays, lin